<a href="https://colab.research.google.com/github/RainaVardhan/ETL_Data_Processor/blob/main/ETL_Data_Processor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [235]:
! git clone https://github.com/RainaVardhan/ETL_Data_Processor.git

fatal: destination path 'ETL_Data_Processor' already exists and is not an empty directory.


In [236]:
! python -m pip install pymongo

In [237]:
import numpy as np
import pandas as pd
import sqlite3

In [238]:
# Connecting to the SQLite Database
conn = sqlite3.connect('movies_and_shows.db')

# Creating the Movies table
conn.execute('''CREATE TABLE IF NOT EXISTS Movies (
                id INTEGER PRIMARY KEY UNIQUE,
                title TEXT,
                description TEXT,
                genre TEXT,
                release_year INTEGER,
                cast TEXT,
                duration TEXT
            )''')

#Creating the Shows table
conn.execute('''CREATE TABLE IF NOT EXISTS Shows (
                id INTEGER PRIMARY KEY UNIQUE,
                title TEXT,
                description TEXT,
                genre TEXT,
                release_year INTEGER,
                cast TEXT,
                seasons TEXT
            )''')

#Creating the MovieRatings table
conn.execute('''CREATE TABLE IF NOT EXISTS MovieRatings (
                id INTEGER PRIMARY KEY,
                movie_id INTEGER,
                title TEXT,
                imdb_id TEXT,
                imdb_rating REAL,
                imdb_votes INTEGER,
                FOREIGN KEY (movie_id) REFERENCES Movies(id)
              )''')

#Creating the ShowRatings table
conn.execute('''CREATE TABLE IF NOT EXISTS ShowRatings (
                id INTEGER PRIMARY KEY,
                show_id INTEGER,
                title TEXT,
                imdb_id TEXT,
                imdb_rating REAL,
                imdb_votes INTEGER,
                FOREIGN KEY (show_id) REFERENCES Shows(id)
              )''')

#Creating the MovieGenres table
conn.execute('''CREATE TABLE IF NOT EXISTS MovieGenres (
                id INTEGER PRIMARY KEY,
                genre TEXT,
                movie_id INTEGER,
                movie_title TEXT,
                FOREIGN KEY (movie_id) REFERENCES Movies(id)
            )''')

#Committing changes to the Database
conn.commit()

#Closing the connection to the Database
conn.close()

In [239]:
#Loading CSV file into pandas DataFrames
netflix = pd.read_csv('/content/netflix_titles.csv')

#Filters the DataFrame so that it only looks at the TV shows
shows = netflix[netflix['type'] == 'TV Show']

#Limits the data to only the first 10 rows
shows = shows.head(10)
print(shows)

#Connecting to the SQLite database
conn = sqlite3.connect('movies_and_shows.db')

#Creating a cursor to execute SQL commands
cursor = conn.cursor()

#Looping through the each row in the DataFrame to insert data into the Shows table in the Database
for index,row in shows.iterrows():
    cursor.execute('''INSERT INTO Shows (title, description, genre, release_year, cast, seasons)
                      VALUES (?, ?, ?, ?, ?, ?)''', (
                        row['title'],
                        row['description'],
                        row['listed_in'],
                        row['release_year'],
                        row['cast'],
                        row['duration']
                  ))

#Committing changes to the Database
conn.commit()

#Closing the connection to the Database
conn.close()

   show_id     type                                title           director  \
1       s2  TV Show                        Blood & Water                NaN   
2       s3  TV Show                            Ganglands    Julien Leclercq   
3       s4  TV Show                Jailbirds New Orleans                NaN   
4       s5  TV Show                         Kota Factory                NaN   
5       s6  TV Show                        Midnight Mass      Mike Flanagan   
8       s9  TV Show        The Great British Baking Show    Andy Devonshire   
10     s11  TV Show  Vendetta: Truth, Lies and The Mafia                NaN   
11     s12  TV Show                     Bangkok Breaking  Kongkiat Komesiri   
14     s15  TV Show      Crime Stories: India Detectives                NaN   
15     s16  TV Show                    Dear White People                NaN   

                                                 cast         country  \
1   Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...    

In [240]:
#Connecting to the SQLite Database
conn = sqlite3.connect('movies_and_shows.db')

#Querrying the data from the Shows table to convert it to a DataFrame (asked chat how to do this)
query = f"SELECT * FROM Shows"
data = pd.read_sql_query(query, conn)

#Closing the connection to the Database
conn.close()

#Exporting the DataFrame to a CSV file
data.to_csv('shows_table.csv', index=False)

In [241]:
import os
import datetime
import json
import pymongo
import pprint

#Connecting the MongoDB with these parameters
host_name = "localhost"
port = "27017"

atlas_cluster_name = "cluster0.xuibg2h"
atlas_default_dbname = "sakila"
atlas_user_name = "ds2002sp23"
atlas_password = "uva1819"

#Creating a MongoDB client using the Atlas connection string
conn_str = {
    "atlas" : f"mongodb+srv://{atlas_user_name}:{atlas_password}@{atlas_cluster_name}.mongodb.net/{atlas_default_dbname}"
}

client = pymongo.MongoClient(conn_str["atlas"])

#Accessing the sample_mflix database and its movies collection
db_name = "sample_mflix"
db = client[db_name]
db.list_collection_names()
collection = db['movies']

#Finding all the entries in the movies collection and turning it into a list
data = collection.find()
data_list = list(data)

#Limits the list to only take the first 10 entries
data_list = data_list[:10]
print(data_list)

[{'_id': ObjectId('573a1390f29313caabcd5ea4'), 'plot': "A District Attorney's outspoken stand on abortion gets him in trouble with the local community.", 'genres': ['Drama'], 'runtime': 62, 'rated': 'APPROVED', 'cast': ['Tyrone Power Sr.', 'Helen Riaume', 'Marie Walcamp', 'Cora Drew'], 'title': 'Where Are My Children?', 'fullplot': 'While prosecuting a physician for the death of a client after an abortion, the district attorney discovers that his wife helped her society friends and the daughter of her maid obtain and pay for abortions from the physician (and was perhaps herself also a client.)', 'languages': ['English'], 'released': datetime.datetime(1916, 5, 1, 0, 0), 'directors': ['Phillips Smalley', 'Lois Weber'], 'writers': ['Lucy Payton (from the story by)', 'Franklin Hall (from the story by)', 'Lois Weber', 'Phillips Smalley'], 'awards': {'wins': 1, 'nominations': 0, 'text': '1 win.'}, 'lastupdated': '2015-09-07 00:51:32.560000000', 'year': 1916, 'imdb': {'rating': 5.9, 'votes': 

In [242]:
from bson import ObjectId

#Connecting to the SQLite Database
conn = sqlite3.connect('movies_and_shows.db')

#Creating a cursor to execute SQL commands
cursor = conn.cursor()

#Looping through the list of movie data from the MongoDB to insert movie data into the Movie table
for movie in data_list:

    #Joining the list of genres and list of cast memebers into single strings separated by commas (asked chat how to do this)
    genres_str = ', '.join(movie.get('genres', []))
    cast = movie.get('cast',[])

    #Checking that cast has valid values
    if isinstance(cast, list):
      cast_str =', '.join(cast)
    else:
      cast = None

    cursor.execute('''INSERT INTO Movies (title, description, genre, release_year, cast, duration)
                      VALUES (?, ?, ?, ?, ?, ?)''', (
                          movie.get('title'),
                          movie.get('plot'),
                          genres_str,
                          movie.get('year'),
                          cast_str,
                          movie.get('runtime')
                  ))

#Committing changes to the Database
conn.commit()

#Closing the connection to the Database
conn.close()

In [243]:
#Connecting to the SQLite Database
conn = sqlite3.connect('movies_and_shows.db')

#Querrying the data from the Movies table to convert it to a DataFrame (asked chat how to do this)
query = f"SELECT * FROM Movies"
data = pd.read_sql_query(query, conn)

#Closing the connection to the Database
conn.close()

#Exporting the DataFrame to a CSV file
data.to_csv('movies_table.csv', index=False)

In [244]:
import requests

#API key details
api_key = '56100165'

#Base URL and parameters for the OMDb API request
def get_api_data(title, api_key):
  base_url = 'http://www.omdbapi.com/'
  params = {
      't': title,
      'apikey': api_key
  }
  try:
    response = requests.get(base_url, params=params)
    response.raise_for_status()
  except requests.exceptions.HTTPError as errh:
    return "An Http Error occurred: " + repr(errh)
  except requests.exceptions.ConnectionError as errc:
    return "An Error Connecting to the API occurred: " + repr(errc)
  except requests.exceptions.Timeout as errt:
    return "A Timeout Error occurred: " + repr(errt)
  except requests.exceptions.RequestException as err:
    return "An Unknown Error occurred: " + repr(err)
  return response.json()

In [245]:
#Connecting to the SQLite Database
conn = sqlite3.connect('movies_and_shows.db')

#Creating a cursor to execute SQL commands
cursor = conn.cursor()

#Getting the titles from Movies and Shows tables
cursor.execute("SELECT title FROM Movies")
fetch = cursor.fetchall()
movies_titles = []
for row in fetch:
  movies_titles.append(row[0])

cursor.execute("SELECT title FROM Shows")
fetch = cursor.fetchall()
shows_titles = []
for row in fetch:
  shows_titles.append(row[0])

#Creating lists to store the movie and show data
movie_data = []
show_data = []

#Getting movie data from OMDb API and adding it to the movie_data list
for title in movies_titles:
    data = get_api_data(title, api_key)
    print(data)
    if data:
        movie_data.append(data)

#Getting show data from OMDb API and adding it to the show_data list
for title in shows_titles:
    data = get_api_data(title, api_key)
    if data:
        show_data.append(data)

#Converting movie_data and show_data lists into DataFrames
movie_ratings = pd.DataFrame(movie_data)
show_ratings = pd.DataFrame(show_data)

#Committing changes to the Database
conn.commit()

#Closing the connection to the Database
conn.close()

{'Title': 'Where Are My Children?', 'Year': '1916', 'Rated': 'Approved', 'Released': '01 May 1916', 'Runtime': '62 min', 'Genre': 'Drama', 'Director': 'Phillips Smalley, Lois Weber', 'Writer': 'Lucy Payton, Franklyn Hall, Lois Weber', 'Actors': 'Tyrone Power Sr., Mrs. Tyrone Power, Marie Walcamp', 'Plot': "A District Attorney's outspoken stand on abortion lands him in trouble with the local community.", 'Language': 'None, English', 'Country': 'United States', 'Awards': '1 win', 'Poster': 'https://m.media-amazon.com/images/M/MV5BM2UxNTNiYjYtYjZkYy00MzI0LWExZWItZDZhOWEzNWFkZGMzXkEyXkFqcGdeQXVyMTk2MzI2Ng@@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.2/10'}], 'Metascore': 'N/A', 'imdbRating': '6.2', 'imdbVotes': '855', 'imdbID': 'tt0007558', 'Type': 'movie', 'DVD': '25 Feb 2020', 'BoxOffice': 'N/A', 'Production': 'N/A', 'Website': 'N/A', 'Response': 'True'}
{'Title': 'Cops', 'Year': '1989–2023', 'Rated': 'TV-14', 'Released': '11 Mar 1989', 'Runtime': '1 mi

In [246]:
#Connecting to the SQLite Database
conn = sqlite3.connect('movies_and_shows.db')

#Creating a cursor to execute SQL commands
cursor = conn.cursor()

#Looping thorugh each row in the movie_ratings DataFrame to insert movie rating data into the MovieRatings table
for index,row in movie_ratings.iterrows():

    #Getting the movie id from the Movies table that corresponds to the title
    cursor.execute("SELECT id FROM Movies WHERE title = ?", (row['Title'],))
    movie_id = cursor.fetchone()

    #Checking if the movie_id exists
    if movie_id:
        movie_id = movie_id[0]
        cursor.execute('''INSERT INTO MovieRatings (movie_id, title, imdb_id, imdb_rating, imdb_votes)
                          VALUES (?, ?, ?, ?, ?)''', (
                              movie_id,
                              row['Title'],
                              row['imdbID'],
                              row['imdbRating'],
                              row['imdbVotes'],
                      ))

#Committing changes to the Database
conn.commit()

#Closing the connection to the Database
conn.close()

In [247]:
#Connecting to the SQLite Database
conn = sqlite3.connect('movies_and_shows.db')

#Creating a cursor to execute SQL commands
cursor = conn.cursor()

#Looping thorugh each row in the show_ratings DataFrame to insert show rating data into the ShowRatings table
for index,row in show_ratings.iterrows():

    #Getting the show id from the Shows table that corresponds to the title
    cursor.execute("SELECT id FROM Shows WHERE title = ?", (row['Title'],))
    show_id = cursor.fetchone()

    # Check if the show_id exists
    if show_id:
        show_id = show_id[0]
        cursor.execute('''INSERT INTO ShowRatings (show_id, title, imdb_id, imdb_rating, imdb_votes)
                          VALUES (?, ?, ?, ?, ?)''', (
                              show_id,
                              row['Title'],
                              row['imdbID'],
                              row['imdbRating'],
                              row['imdbVotes'],
                      ))

#Committing changes to the Database
conn.commit()

#Closing the connection to the Database
conn.close()

In [248]:
#Connecting to the SQLite Database
conn = sqlite3.connect('movies_and_shows.db')

#Querrying the data from the MovieRatings table to convert it to a DataFrame (asked chat how to do this)
query = f"SELECT * FROM MovieRatings"
data = pd.read_sql_query(query, conn)

#Closing the connection to the Database
conn.close()

#Exporting the DataFrame to a CSV file
data.to_csv('movies_ratings_table.csv', index=False)

In [249]:
#Connecting to the SQLite Database
conn = sqlite3.connect('movies_and_shows.db')

#Querrying the data from the ShowRatings table to convert it to a DataFrame (asked chat how to do this)
query = f"SELECT * FROM ShowRatings"
data = pd.read_sql_query(query, conn)

#Closing the connection to the Database
conn.close()

#Exporting the DataFrame to a CSV file
data.to_csv('show_ratings_table.csv', index=False)

In [250]:
# Connecting to the SQLite Database
conn = sqlite3.connect('movies_and_shows.db')

# Creating a cursor to execute SQL commands
cursor = conn.cursor()

# Looping through each row in the combined_data DataFrame to store data in the dictionary
for index, row in movie_ratings.iterrows():
    title = row['Title']

    # Splitting the genres into a list
    genres = row['Genre']
    if isinstance(genres, str):
        genres = genres.split(', ')
    else:
        genres = []

    # Finding the movie id from the Movies table that match the title
    cursor.execute("SELECT id FROM Movies WHERE title = ?", (title,))
    movie_id = cursor.fetchone()

    # Checking if movie_id is valid values
    if movie_id:
        movie_id = movie_id[0]
    else:
        movie_id = None

    # Looping through the genres to update the dictionary to show the movie titles and ids based on the genre
    for genre in genres:
        if movie_id:
            cursor.execute('''INSERT INTO MovieGenres (genre, movie_id, movie_title)
                              VALUES (?, ?, ?)''', (
                                  genre,
                                  movie_id,
                                  title,
                          ))

# Committing changes to the Database
conn.commit()

# Closing the connection to the Database
conn.close()

In [251]:
#Connecting to the SQLite Database
conn = sqlite3.connect('movies_and_shows.db')

#Querrying the data from the ShowRatings table to convert it to a DataFrame (asked chat how to do this)
query = f"SELECT * FROM MovieGenres"
data = pd.read_sql_query(query, conn)

#Closing the connection to the Database
conn.close()

#Exporting the DataFrame to a CSV file
data.to_csv('movie_genres_table.csv', index=False)

In [252]:
!pip install pyMysql

In [253]:
import pymysql
import warnings

warnings.filterwarnings('ignore')

#MySQL Database connection parameters
host = "datatbase.ds2002.org"
user = 'cdw3pe'
pwd = "cdw3pe!"
db_name = "cdw3pe"

#Connecting to the MySQL Database
conn = pymysql.connect(host=host, user=user, password=pwd, database=db_name)

#Creating a cursor to execute SQL commands
cursor = conn.cursor()

#Selecting the genre to find the average IMDb rating and the count of movies for each genre
sql_query = """
SELECT movie_genre.genre, AVG(movie_rating.imdb_rating) AS avg_rating, COUNT(movie.id) AS movie_count
FROM MovieGenres movie_genre
JOIN Movies movie ON movie_genre.movie_id = movie.id
JOIN MovieRatings movie_rating ON movie.id = movie_rating.movie_id
GROUP BY movie_genre.genre;
"""

#Executing and storing the SQL query
cursor.execute(sql_query)
results = cursor.fetchall()

#Printing the results from the SQL query
for row in results:
    genre = row[0]
    avg_rating = row[1]
    movie_count = row[2]
    print(f"Genre: {genre}, Average Rating: {avg_rating}, Movie Count: {movie_count}")

#Closing the cursor and connection
conn.close()

Genre: Drama, Average Rating: 6.9, Movie Count: 5
Genre: Action, Average Rating: 6.9, Movie Count: 1
Genre: Crime, Average Rating: 6.9, Movie Count: 1
Genre: Reality-TV, Average Rating: 6.9, Movie Count: 1
Genre: Romance, Average Rating: 7.033333333333334, Movie Count: 3
Genre: Comedy, Average Rating: 6.8, Movie Count: 1
Genre: Musical, Average Rating: 6.8, Movie Count: 1
Genre: Family, Average Rating: 6.8, Movie Count: 1
Genre: History, Average Rating: 6.8, Movie Count: 1
